## Preliminaries

### Libraries

In [1]:
import os
import glob
import sys
import shutil
import math
import zipfile

<br>

### Paths

In [2]:
root = os.path.split(os.getcwd())[0]
root

'J:\\Library\\projects\\augmentation'

In [3]:
sys.path.append( root )

<br>

### Local Imports

After stteing-up appropriate paths, local packages, etc., can be imported.

In [4]:
import src.cfg.cfg as cfg

<br>
<br>

## Images


### The path to the images

In [5]:
variables = cfg.Cfg().variables()
path = os.path.join(root, variables['target']['images']['directory'])

<br>

### The list of images

In [6]:
images = glob.glob(os.path.join(path, '*.png'))

In [7]:
type(images)

list

<br>

### Assigning a unique number to each image

Options:

```python
map(list.__add__, images, list(range(len(images))))

[ list(x) for x in zip(images, list(range(len(images)))) ]

```

<br>

In [8]:
unique_id = list(range(len(images)))

In [9]:
images_ = [ list(x) for x in zip(images, unique_id) ]

In [10]:
images_[0][0]

'J:\\Library\\projects\\augmentation\\images\\ISIC_0000021_downsampled-000.png'

<br>

## Splitting

### Number of images per directory

In [11]:
N = 320

<br>

### Top directory of splits

In [12]:
splits = os.path.join(root, 'splits')

In [13]:
for i in glob.glob(os.path.join(splits, '*', '*')):
    os.remove(i)

In [14]:
for i in glob.glob(os.path.join(splits, '*')):
    os.removedirs(i)

In [15]:
if not os.path.exists(splits):
    os.mkdir(splits)

<br>

### Image directory function

In [16]:
def dermacating(image_name, image_index):
    
    # create new directory if necessary
    subdirectory = os.path.join(splits, '{0:03d}'.format( math.floor(image_index / N) ) )
    print(subdirectory)
    
    if not os.path.exists(subdirectory):
        os.mkdir(subdirectory)
        

    # copy file to subdirectory    
    base = os.path.basename(image_name)
    shutil.copy(image_name, subdirectory)
    

<br>

### Copy images to assigned split directories

In [17]:
 outcomes = [dermacating(image_name, image_index) for image_name, image_index in images_]

J:\Library\projects\augmentation\splits\000
J:\Library\projects\augmentation\splits\000
J:\Library\projects\augmentation\splits\000
J:\Library\projects\augmentation\splits\000
J:\Library\projects\augmentation\splits\000
J:\Library\projects\augmentation\splits\000
J:\Library\projects\augmentation\splits\000
J:\Library\projects\augmentation\splits\000
J:\Library\projects\augmentation\splits\000
J:\Library\projects\augmentation\splits\000
J:\Library\projects\augmentation\splits\000
J:\Library\projects\augmentation\splits\000
J:\Library\projects\augmentation\splits\000
J:\Library\projects\augmentation\splits\000
J:\Library\projects\augmentation\splits\000
J:\Library\projects\augmentation\splits\000
J:\Library\projects\augmentation\splits\000
J:\Library\projects\augmentation\splits\000
J:\Library\projects\augmentation\splits\000
J:\Library\projects\augmentation\splits\000
J:\Library\projects\augmentation\splits\000
J:\Library\projects\augmentation\splits\000
J:\Library\projects\augmentation

<br>
<br>

## Zipping

### Re-cap

In [18]:
splits

'J:\\Library\\projects\\augmentation\\splits'

In [19]:
root

'J:\\Library\\projects\\augmentation'

<br>

### A directory for the zips

In [20]:
zips = os.path.join(root, 'zips')
zips

'J:\\Library\\projects\\augmentation\\zips'

In [21]:
for i in glob.glob(os.path.join(zips, '*')):
    os.remove(i)

In [22]:
if not os.path.exists(zips):
        os.mkdir(zips)

<br>

### Zipping function

In [23]:
def zipper(dir):
    
    # The archive name will be the raw directory name
    arcname = os.path.basename(dir)
    
    # Open ...
    z = zipfile.ZipFile(os.path.join(zips, arcname + '.zip'), 'x')
    
    # The list of images in 'dir'
    images = glob.glob(os.path.join(dir, '*.png'))
    
    # Write each image in the list to arcname
    [z.write(filename=images[i], arcname=os.path.basename(images[i]), compress_type=zipfile.ZIP_DEFLATED) for i in range(len(images))]
    
    z.close()
    

<br>

### The list of directories of ...

In [24]:
directories_of_splits = glob.glob(os.path.join(splits, '*'))
directories_of_splits

['J:\\Library\\projects\\augmentation\\splits\\000',
 'J:\\Library\\projects\\augmentation\\splits\\001',
 'J:\\Library\\projects\\augmentation\\splits\\002',
 'J:\\Library\\projects\\augmentation\\splits\\003']

<br>

### Hence, zip each directory

In [25]:
[ zipper(i) for i in directories_of_splits ]

[None, None, None, None]